In [22]:
from selenium import webdriver 
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from bs4 import BeautifulSoup
import re
import pandas as pd
from tqdm import tqdm
import time

In [23]:
from urllib.request import Request, urlopen
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
from urllib.parse import urljoin

In [24]:
chrome_options = webdriver.ChromeOptions()

In [25]:
url = 'https://www.starbucks.co.kr/store/store_map.do'
driver = webdriver.Chrome(service=Service('../Selenium/driver/chromedriver'), options = chrome_options)
driver.get(url)

In [26]:
driver.find_element(By.CLASS_NAME, 'loca_search').click() ## 지역검색 클릭

In [27]:
driver.find_elements(By.CLASS_NAME, 'set_sido_cd_btn')[0].click() ## 서울 클릭

In [28]:
driver.find_elements(By.CLASS_NAME, 'set_gugun_cd_btn')[0].click() ## 전체 클릭 

In [29]:
req = driver.page_source
soup = BeautifulSoup(req, "html.parser") ## 현재 페이지 html 가져오기

In [30]:
content = soup.find("div", id = "mCSB_3_container")
contents = content.find_all("li") ## 원하는 매장별 html 위치 접근하기 

In [31]:
gu = soup.find('ul', 'gugun_arae_box')

In [53]:
gu_names = []
for i in range(len(gu('a'))):
    gu_name = gu('a')[i].text
    gu_names.append(gu_name)
gu_names = gu_names[1:]

['강남구',
 '강동구',
 '강북구',
 '강서구',
 '관악구',
 '광진구',
 '구로구',
 '금천구',
 '노원구',
 '도봉구',
 '동대문구',
 '동작구',
 '마포구',
 '서대문구',
 '서초구',
 '성동구',
 '성북구',
 '송파구',
 '양천구',
 '영등포구',
 '용산구',
 '은평구',
 '종로구',
 '중구',
 '중랑구']

In [21]:
starbucksList = []
for content in contents:
    info = content.find(class_='result_details').get_text()
    info = info.split(" ")
    
    gu   = info[0] + " " + info[1]
    addr = info[0] + " " + info[1] + " " + info[2] + " " + info[3]
    name = content['data-name']
    lat  = content['data-lat']
    long = content['data-long']
    
    each = {"매장이름" : name,
           "주소" : addr,
           "구" : gu,
           "위도" : lat,
           "경도" : long,
           "브랜드" : "starbucks"}
    starbucksList.append(each)              
    
## 데이터 크롤링 한 후 데이터 프레임 만들기 
## 딕셔너리 형태로 데이터 만든 후 리스트에 넣고 딕셔너리 데이터를 갖고 있는 리스트를 판다스 데이터 프레임에 넣어주면 
## 딕셔너리 key값이 columns , 데이터 프레임 value 값이 value로 생성됨 
    

df_starbucks = pd.DataFrame(starbucksList)
df_starbucks

,매장이름,주소,구,위도,경도,브랜드
0,역삼아레나빌딩,서울특별시 강남구 언주로 425,서울특별시 강남구,37.501087,127.043069,starbucks
1,논현역사거리,서울특별시 강남구 강남대로 538,서울특별시 강남구,37.510178,127.022223,starbucks
2,신사역성일빌딩,서울특별시 강남구 강남대로 584,서울특별시 강남구,37.5139309,127.0206057,starbucks
3,국기원사거리,서울특별시 강남구 테헤란로 125,서울특별시 강남구,37.499517,127.031495,starbucks
4,대치재경빌딩R,서울특별시 강남구 남부순환로 2947,서울특별시 강남구,37.494668,127.062583,starbucks
...,...,...,...,...,...,...
588,사가정역,서울특별시 중랑구 면목로 3101522-3232,서울특별시 중랑구,37.579594,127.087966,starbucks
589,상봉역,서울특별시 중랑구 망우로 307,서울특별시 중랑구,37.59689,127.08647,starbucks
590,묵동,서울특별시 중랑구 동일로 952,서울특별시 중랑구,37.615368,127.076633,starbucks
591,양원역,서울특별시 중랑구 양원역로10길 3,서울특별시 중랑구,37.6066536267232,127.106359790053,starbucks
